# Занятие 9: Изучение работы методов контроля температуры в молекулярной динамике

### Лев Мазаев, мАДМБ 18

In [1]:
# Visualization
import nglview as nv

# Working with Molecules
from rdkit import Chem
from rdkit.Chem import AllChem

_ColormakerRegistry()

RDKit WARNING: [15:30:50] Enabling RDKit 2019.09.2 jupyter extensions


### 1. Подготовка файла координат и файла топологии

Скачаем файл координат:

In [2]:
%%bash
wget -q -nc http://kodomo.fbb.msu.ru/FBB/year_08/term6/etane.gro
mv etane.gro et.gro

У нас есть шаблон файла топологии:

```
#include "/usr/share/gromacs/top/oplsaa.ff/forcefield.itp" 

[ moleculetype ]
; Name            nrexcl
et            3

[ atoms ]
;   nr  type  resnr  residue  atom   cgnr     charge       mass
    1   CX      1    ETH      C1      1    -0.189      12.01
    2   CX      1    ETH      C2      2    -0.155      12.01
    3   HX      1    ETH      H1      3     0.0059       1.008
    4   HX      1    ETH      H2      4     0.0059       1.008
    5   HX      1    ETH      H3      5     0.0059       1.008
    6   HX      1    ETH      H4      6     0.0056       1.008
    7   HX      1    ETH      H5      7     0.0056       1.008
    8   HX      1    ETH      H6      8     0.0056       1.008

[ bonds ]
;  ai    aj funct  b0       kb
     1   2   1  
     1   3   1
     1   4   1  
     1   5   1  
........... 
надо дописать

[ angles ]
;  ai    aj    ak funct  phi0   kphi
;around c1
    3     1     4     1  
    4     1     5     1  
    3     1     5     1  
    2     1     3     1  
    2     1     4     1  
    2     1     5     1  
;around c2
........... 
надо дописать 

[ dihedrals ]
;  ai    aj    ak    al funct  
    3    1     2     6      1  
    3    1     2     7      1 
    3    1     2     8      1  
    4    1     2     6      1  
 ........... 
надо дописать 

[ pairs ]
; список атомов 1-4
;  ai    aj funct
   3  6
   3  7
   3  8
   4  6
   4  7
   4  8
   5  6
   5  7
   5  8

[ System ]
; any text here
first one
[ molecules ]
;Name count
 et    1
```

Нужно его дополнить, чтобы получить правильный файл. В пакете gromacs есть функция [x2top](http://manual.gromacs.org/documentation/5.1.1/onlinehelp/gmx-x2top.html), которая генерирует примитивный файл топологии на основе структуры. Можно этим воспользоваться:

```bash
gmx x2top -f et.gro -o preet.top -ff oplsaa
```

In [3]:
! cat preet.top

;
;	File 'preet.top' was generated
;	By user: leo (1000)
;	On host: MS7922
;	At date: Mon Dec 16 15:05:28 2019
;
;	This is a include topology file
;
;	Created by:
;	                     :-) GROMACS - gmx x2top, 2019.4 (-:
;	
;	Executable:   /usr/local/gromacs/bin/gmx
;	Data prefix:  /usr/local/gromacs
;	Working dir:  /home/leo/ME/HSE/Chem/9 - Temperature in Molecular Dynamics
;	Command line:
;	  gmx x2top -f et.gro -o preet.top -ff oplsaa
;	Force field was read from the standard GROMACS share directory.
;

; Include forcefield parameters
#include "oplsaa.ff/forcefield.itp"

[ moleculetype ]
; Name            nrexcl
ICE                 3

[ atoms ]
;   nr       type  resnr residue  atom   cgnr     charge       mass  typeB    chargeB      massB
     1   opls_157      1    ETH     C1      1      -0.18     12.011
     2   opls_157      1    ETH     C2      1      -0.18     12.011
     3   opls_140      1    ETH     H1      1       0.06      1.008
     4   opls_140      1    ETH     H2     

Типы атомов в oplsaa.ff: атомы opls с 135 по 140 - специально для алканов (т. е. и этана). То есть для водорода `x2top` предложила правильный тип **opls_140 - alkane H**, а вот **opls_157 - all-atom C: CH3 & CH2, alcohols** пожалуй не самый удачный. Заменим его на **opls_135 - alkane CH3**.

Также нам нужно дописать углы и связи в исходной топологии, в чем нам поможет результат `x2top`, а также `rdkit`. Визуализируем этан:

In [11]:
ethane = Chem.MolFromSmiles('CC')
ethane.Compute2DCoords()
ethane = Chem.AddHs(ethane)
AllChem.EmbedMolecule(ethane)
AllChem.MMFFOptimizeMolecule(ethane, maxIters=500, nonBondedThresh=200)
v = nv.show_rdkit(ethane)
v

NGLWidget()

Химические связи (bonds):

In [21]:
for b in ethane.GetBonds():
    print(b.GetBeginAtomIdx() + 1, b.GetEndAtomIdx() + 1)

1 2
1 3
1 4
1 5
2 6
2 7
2 8


Углы (angles):

In [23]:
for b1 in ethane.GetBonds():
    for b2 in ethane.GetBonds():
        if b1.GetBeginAtomIdx() == b2.GetBeginAtomIdx() and b1.GetIdx() != b2.GetIdx():
            print(b1.GetEndAtomIdx() + 1, b1.GetBeginAtomIdx() + 1, b2.GetEndAtomIdx() + 1)

2 1 3
2 1 4
2 1 5
3 1 2
3 1 4
3 1 5
4 1 2
4 1 3
4 1 5
5 1 2
5 1 3
5 1 4
6 2 7
6 2 8
7 2 6
7 2 8
8 2 6
8 2 7
